In [1]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings('ignore')

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data_main = requests.get(standings_url)

In [4]:
soup = BeautifulSoup(data_main.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [5]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [53]:
team_name = team_urls[0].split("/")[-1].replace("-Stats", "").replace("-", " ")

In [54]:
team_name

'Manchester City'

In [6]:
data_city = requests.get(team_urls[0])

In [7]:
matches = pd.read_html(data_city.text, match="Scores & Fixtures")[0]

In [8]:
a = matches.copy()

In [9]:
def grab_data(pattern,data_main ,match):
    soup = BeautifulSoup(data_main.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/'+pattern+'/' in l]
    data_in = requests.get(f"https://fbref.com{links[0]}")
    df = pd.read_html(data_in.text, match=match)[0]
    print(f"Got {match} data sucessfully")
    return df

In [10]:
def clean_data(df,renamed_col,df_name):
    df = df.copy()
    df.columns = df.columns.droplevel()
    df.columns = renamed_col
    df.drop(['Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent','Match Report'],axis=1,inplace=True)
    print(f"Shape  {df_name} : " ,df.shape)
    return df

In [30]:
shooting = grab_data(pattern='shooting',data_main=data_city ,match='Shooting')

Got Shooting data sucessfully


In [12]:
    Shooting_common_col = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent']
    Shooting_Standard_col = ['Standard_'+col for col in   ['Gls', 'Sh', 'SoT', 'SoT%', 'G/Sh', 'G/SoT', 'Dist', 'FK','PK', 'PKatt']] 
    Shooting_Expected_col =  ['Expected_'+col for col in ['xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG']] 
    Shooting_renamed_col = Shooting_common_col + Shooting_Standard_col + Shooting_Expected_col + ['Match Report']

In [31]:
#b =clean_shooting_data(Shooting=shooting)
b = clean_data(df=shooting,renamed_col=Shooting_renamed_col,df_name='Shooting')

Shape  Shooting :  (1, 16)


In [65]:
GSA = grab_data(pattern='gca',data_main=data_city ,match="Goal and Shot")

Got Goal and Shot data sucessfully


In [16]:
    GSA_common_col = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent']
    GSA_SCA_Types_col = ['SCA_Types_'+col for col in   ['SCA', 'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def']] 
    GSA_GCA_Types_col =  ['GCA_Types_'+col for col in ['GCA', 'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def']] 
    GSA_renamed_col= GSA_common_col + GSA_SCA_Types_col + GSA_GCA_Types_col + ['Match Report']

In [66]:
#clean_GSA_data(Goal_and_Shot=GSA,renamed_col=GSA_renamed_col)
c = clean_data(df=GSA,renamed_col=GSA_renamed_col,df_name='GSA')

Shape  GSA :  (59, 15)


In [70]:
DEF = grab_data(pattern='defense',data_main=data_city ,match="Defensive Actions")

Got Defensive Actions data sucessfully


In [19]:
    Defensive_Actions_common_col = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent']
    Defensive_Actions_Tackles_col = ['Tackles_'+col for col in   ['Tkl', 'TklW', 'Def_3rd', 'Mid_3rd', 'Att_3rd']] 
    Defensive_Actions_Vs_Dribbles_col =  ['Vs_Dribbles_'+col for col in [ 'Tkl','Att', 'Tkl%', 'Past',]]
    Defensive_Actions_Pressures_col =  ['Pressures_'+col for col in ['Press', 'Succ', '%', 'Def_3rd', 'Mid_3rd','Att_3rd']]
    Defensive_Actions_Blocks_col =  ['Blocks_'+col for col in ['Blocks', 'Sh', 'ShSv', 'Pass']]
    Defensive_Actions_Def_col =  ['Def_'+col for col in ['Int', 'Tkl+Int', 'Clr','Err']]
    Defensive_Actions_renamed_col= Defensive_Actions_common_col + Defensive_Actions_Tackles_col + Defensive_Actions_Vs_Dribbles_col + Defensive_Actions_Pressures_col + Defensive_Actions_Blocks_col + Defensive_Actions_Def_col + ['Match Report']

In [71]:
d = clean_data(df=DEF,renamed_col=Defensive_Actions_renamed_col,df_name='Defensive Actions')

Shape  Defensive Actions :  (59, 24)


In [75]:
goalie = grab_data(pattern='keeper', data_main=data_city, match="Goalkeeping")

Got Goalkeeping data sucessfully


In [22]:
    Goalkeeping_cols=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent']
    gk_abrevations = ['GK_Perf_','GK_Penalty_','GK_Launch_','GK_Passes_','Gk_Goal_Kk_','GK_Crosses_','Gk_Sweeper_']
    GK_listings = [
        ['SoTA', 'GA', 'Saves', 'Save%', 'CS', 'PSxG', 'PSxG+/-'],#GK_Perf_
        ['PKatt', 'PKA', 'PKsv', 'PKm'],#GK_Penalty_
        ['Cmp', 'Att', 'Cmp%'],#GK_Launch_
        ['Att', 'Thr','Launch%', 'AvgLen'],#GK_Passes_
        ['Att', 'Launch%', 'AvgLen'],#Gk_Goal_Kk_
        ['Opp', 'Stp', 'Stp%'],#GK_Crosses_
        ['#OPA', 'AvgDist'],#Gk_Sweeper_
    ]

    for abv,listings in zip(gk_abrevations,GK_listings):
        new_cols = [abv+col for col in listings]
        Goalkeeping_cols.extend(new_cols)
    Goalkeeping_renamed_col = Goalkeeping_cols + ['Match Report']


In [76]:
e = clean_data(df=goalie,renamed_col=Goalkeeping_renamed_col,df_name='Goalkeeping')

Shape  Goalkeeping :  (59, 27)


In [38]:
passing = grab_data(pattern='passing', data_main=data_city, match="Passing")

Got Passing data sucessfully


In [25]:
    Passing_cols=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent']
    passing_abrevations = ['Passing_Total_','Passing_Short_','Passing_Medium_','Passing_Long_','Passing_']
    passing_listings = [
        ['Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist'],#Passing_Total_
        ['Cmp', 'Att','Cmp%'],#Passing_Short_
        ['Cmp', 'Att', 'Cmp%'],#'Passing_Medium_'
        ['Cmp', 'Att', 'Cmp%'],#Passing_Long_
        ['Ast', 'xA', 'KP','1/3', 'PPA', 'CrsPA', 'Prog'],#Passing_
    ]

    for abv,listings in zip(passing_abrevations,passing_listings):
        new_cols = [abv+col for col in listings]
        Passing_cols.extend(new_cols)
    Passing_renamed_col = Passing_cols + ['Match Report']


In [39]:
passing

For Manchester City                                                      \
                 Date Time Comp Round Day Venue  Result   GF  GA Opponent   
0                 NaN  NaN  NaN   NaN NaN   NaN  41-9-8  150  50      NaN   

   Total                               Short              Medium               \
     Cmp    Att  Cmp% TotDist PrgDist    Cmp    Att  Cmp%    Cmp    Att  Cmp%   
0  31037  35247  88.1  590100  156784  13000  14018  92.7  13648  14862  91.8   

   Long             Unnamed: 24_level_0 Unnamed: 25_level_0  \
    Cmp   Att  Cmp%                 Ast                  xA   
0  4036  5458  73.9                 102                77.3   

  Unnamed: 26_level_0 Unnamed: 27_level_0 Unnamed: 28_level_0  \
                   KP                 1/3                 PPA   
0                 676                2552                 726   

  Unnamed: 29_level_0 Unnamed: 30_level_0 Unnamed: 31_level_0  
                CrsPA                Prog        Match Report  
0                 100                2331                 NaN

In [26]:
f = clean_data(df=passing, renamed_col = Passing_renamed_col ,df_name='Passing')

Shape  Passing :  (1, 22)


In [85]:
pass_types = grab_data(pattern='passing_types', data_main=data_city, match="Pass Types")

Got Pass Types data sucessfully


In [86]:
pass_types

For Manchester City                                                     \
                  Date   Time              Comp                Round  Day   
0           2021-08-07  17:15  Community Shield  FA Community Shield  Sat   
1           2021-08-15  16:30    Premier League          Matchweek 1  Sun   
2           2021-08-21  15:00    Premier League          Matchweek 2  Sat   
3           2021-08-28  12:30    Premier League          Matchweek 3  Sat   
4           2021-09-11  15:00    Premier League          Matchweek 4  Sat   
5           2021-09-15  20:00      Champions Lg          Group stage  Wed   
6           2021-09-18  15:00    Premier League          Matchweek 5  Sat   
7           2021-09-21  19:45           EFL Cup          Third round  Tue   
8           2021-09-25  12:30    Premier League          Matchweek 6  Sat   
9           2021-09-28  21:00      Champions Lg          Group stage  Tue   
10          2021-10-03  16:30    Premier League          Matchweek 7  Sun   
11          2021-10-16  15:00    Premier League          Matchweek 8  Sat   
12          2021-10-19  18:45      Champions Lg          Group stage  Tue   
13          2021-10-23  17:30    Premier League          Matchweek 9  Sat   
14          2021-10-27  19:45           EFL Cup         Fourth round  Wed   
15          2021-10-30  15:00    Premier League         Matchweek 10  Sat   
16          2021-11-03  20:00      Champions Lg          Group stage  Wed   
17          2021-11-06  12:30    Premier League         Matchweek 11  Sat   
18          2021-11-21  14:00    Premier League         Matchweek 12  Sun   
19          2021-11-24  20:00      Champions Lg          Group stage  Wed   
20          2021-11-28  14:00    Premier League         Matchweek 13  Sun   
21          2021-12-01  20:15    Premier League         Matchweek 14  Wed   
22          2021-12-04  17:30    Premier League         Matchweek 15  Sat   
23          2021-12-07  18:45      Champions Lg          Group stage  Tue   
24          2021-12-11  12:30    Premier League         Matchweek 16  Sat   
25          2021-12-14  20:00    Premier League         Matchweek 17  Tue   
26          2021-12-19  14:15    Premier League         Matchweek 18  Sun   
27          2021-12-26  15:00    Premier League         Matchweek 19  Sun   
28          2021-12-29  20:15    Premier League         Matchweek 20  Wed   
29          2022-01-01  12:30    Premier League         Matchweek 21  Sat   
30          2022-01-07  20:00            FA Cup   Third round proper  Fri   
31          2022-01-15  12:30    Premier League         Matchweek 22  Sat   
32          2022-01-22  17:30    Premier League         Matchweek 23  Sat   
33          2022-02-05  15:00            FA Cup  Fourth round proper  Sat   
34          2022-02-09  19:45    Premier League         Matchweek 24  Wed   
35          2022-02-12  17:30    Premier League         Matchweek 25  Sat   
36          2022-02-15  20:00      Champions Lg          Round of 16  Tue   
37          2022-02-19  17:30    Premier League         Matchweek 26  Sat   
38          2022-02-26  17:30    Premier League         Matchweek 27  Sat   
39          2022-03-01  19:15            FA Cup   Fifth round proper  Tue   
40          2022-03-06  16:30    Premier League         Matchweek 28  Sun   
41          2022-03-09  20:00      Champions Lg          Round of 16  Wed   
42          2022-03-14  20:00    Premier League         Matchweek 29  Mon   
43          2022-03-20  15:00            FA Cup       Quarter-finals  Sun   
44          2022-04-02  15:00    Premier League         Matchweek 31  Sat   
45          2022-04-05  20:00      Champions Lg       Quarter-finals  Tue   
46          2022-04-10  16:30    Premier League         Matchweek 32  Sun   
47          2022-04-13  21:00      Champions Lg       Quarter-finals  Wed   
48          2022-04-16  15:30            FA Cup          Semi-finals  Sat   
49          2022-04-20  20:00    Premier League         Matchweek 30  Wed   

In [42]:
    PassingType_cols=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent','Pass_Att']
    passingType_abrevations = ['PassType_','Corner_','Pass_Height_','PassWith_','Pass_Outome_']
    passingType_listings = [
        ['Live', 'Dead', 'FK', 'TB', 'Press', 'Sw', 'Crs','CK'],#PassType_
        ['In', 'Out', 'Str'],#Corner_
        [ 'Ground', 'Low', 'High'],#Pass_Height_
        ['Left', 'Right','Head', 'TI','Other'],#PassWith_
        ['Cmp', 'Off', 'Out', 'Int', 'Blocks'],#Pass_Outome_
    ]

    for abv,listings in zip(passingType_abrevations,passingType_listings):
        new_cols = [abv+col for col in listings]
        PassingType_cols.extend(new_cols)
    PassingType_renamed_col = PassingType_cols + ['Match Report']

In [43]:
g = clean_data(df=pass_types,renamed_col=PassingType_renamed_col,df_name='PassingType')

Shape  PassingType :  (59, 26)


In [91]:
possession = grab_data(pattern='possession', data_main=data_city, match="Possession")

Got Possession data sucessfully


In [45]:
    Possession_cols=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']
    Possession_abrevations = ['Touches_','Dribbles_','Carries_','Receiving_',]
    Possession_listings = [
        ['Touches', 'Def_Pen', 'Def_3rd', 'Mid_3rd', 'Att_3rd', 'Att_Pen', 'Live'],#Touches_
        ['Succ', 'Att', 'Succ%', '#Pl', 'Megs'],#Dribbles_
        [ 'Carries', 'TotDist', 'PrgDist', 'Prog', '1/3', 'CPA', 'Mis', 'Dis'],#Carries_
        ['Targ', 'Rec', 'Rec%', 'Prog'],#Receiving_
    ]

    for abv,listings in zip(Possession_abrevations,Possession_listings):
        new_cols = [abv+col for col in listings]
        Possession_cols.extend(new_cols)
    Possession_renamed_col = Possession_cols + ['Match Report']

In [92]:
h = clean_data(df=possession,renamed_col=Possession_renamed_col,df_name='Possession')

Shape  Possession :  (59, 26)


In [94]:
Misc_data = grab_data(pattern='misc', data_main=data_city, match="Miscellaneous Stats")

Got Miscellaneous Stats data sucessfully


In [48]:
    Miscellaneous_Stats_cols=['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent',]
    Miscellaneous_Stats_abrevations = ['Misc_Stats_','Misc_Stats_Aerial_Duels']
    Miscellaneous_Stats_listings = [
        ['CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int','TklW', 'PKwon', 'PKcon', 'OG', 'Recov'],#Misc_Stats_
        ['Won', 'Lost', 'Won%'],#Misc_Stats_Aerial_Duels
    ]

    for abv,listings in zip(Miscellaneous_Stats_abrevations,Miscellaneous_Stats_listings):
        new_cols = [abv+col for col in listings]
        Miscellaneous_Stats_cols.extend(new_cols)
    Miscellaneous_Stats_renamed_col = Miscellaneous_Stats_cols + ['Match Report']

In [95]:
i = clean_data(df=Misc_data,renamed_col=Miscellaneous_Stats_renamed_col ,df_name='Miscellaneous_Stats')

Shape  Miscellaneous_Stats :  (59, 17)


In [55]:
a['Team'] = team_name 

In [56]:
a

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,Manchester City
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.1,0.4,63,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,Manchester City
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,NaN,NaN,79,30959.0,Kevin De Bruyne,4-3-3,Robert Jones,Match Report,NaN,Manchester City
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,1.7,0.3,60,40036.0,Rúben Dias,4-3-3,Michael Oliver,Match Report,NaN,Manchester City
9,2021-09-28,21:00,Champions Lg,Group stage,Tue,Away,L,0,2,fr Paris S-G,1.9,0.8,54,37350.0,Rúben Dias,4-3-3,Carlos del Cerro,Match Report,NaN,Manchester City


In [57]:
b['Team'] = team_name 

In [60]:
b.drop(['Date'],axis=1,inplace=True)

In [61]:
b

,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Team
0,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,Manchester City


In [62]:
ab = a.merge(b, on="Team")

In [63]:
ab

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.1,0.4,63,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,NaN,NaN,79,30959.0,Kevin De Bruyne,4-3-3,Robert Jones,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,1.7,0.3,60,40036.0,Rúben Dias,4-3-3,Michael Oliver,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4
9,2021-09-28,21:00,Champions Lg,Group stage,Tue,Away,L,0,2,fr Paris S-G,1.9,0.8,54,37350.0,Rúben Dias,4-3-3,Carlos del Cerro,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4


In [67]:
abc = ab.merge(c, on="Date")

In [68]:
abc

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,41.0,39.0,1.0,1.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,38.0,30.0,2.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,25.0,18.0,1.0,1.0,3.0,0.0,2.0,7.0,3.0,1.0,0.0,2.0,0.0,1.0
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.1,0.4,63,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,26.0,21.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,NaN,NaN,79,30959.0,Kevin De Bruyne,4-3-3,Robert Jones,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,1.7,0.3,60,40036.0,Rúben Dias,4-3-3,Michael Oliver,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,26.0,17.0,3.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
9,2021-09-28,21:00,Champions Lg,Group stage,Tue,Away,L,0,2,fr Paris S-G,1.9,0.8,54,37350.0,Rúben Dias,4-3-3,Carlos del Cerro,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,33.0,27.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
d

,Date,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err
0,2021-08-07,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
1,2021-08-15,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0
2,2021-08-21,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0
3,2021-08-28,11.0,8,2.0,7.0,2.0,5.0,10.0,50.0,5.0,86.0,34.0,39.5,16.0,44.0,26.0,5.0,1.0,0.0,4.0,11,NaN,7.0,1.0
4,2021-09-11,13.0,7,6.0,4.0,3.0,2.0,7.0,28.6,5.0,114.0,27.0,23.7,27.0,50.0,37.0,9.0,2.0,0.0,7.0,12,NaN,9.0,0.0
5,2021-09-15,17.0,8,5.0,10.0,2.0,10.0,14.0,71.4,4.0,134.0,38.0,28.4,32.0,70.0,32.0,15.0,4.0,0.0,11.0,15,NaN,12.0,0.0
6,2021-09-18,17.0,10,11.0,6.0,0.0,4.0,13.0,30.8,9.0,121.0,31.0,25.6,33.0,62.0,26.0,14.0,6.0,0.0,8.0,14,NaN,22.0,0.0
7,2021-09-21,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
8,2021-09-25,19.0,15,5.0,8.0,6.0,5.0,16.0,31.3,11.0,163.0,52.0,31.9,26.0,68.0,69.0,15.0,5.0,0.0,10.0,10,NaN,24.0,0.0
9,2021-09-28,23.0,14,9.0,11.0,3.0,10.0,27.0,37.0,17.0,150.0,29.0,19.3,33.0,66.0,51.0,9.0,0.0,0.0,9.0,21,NaN,3.0,0.0


In [72]:
abcd = abc.merge(d, on="Date")

In [73]:
abcd

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,41.0,39.0,1.0,1.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,11.0,8,2.0,7.0,2.0,5.0,10.0,50.0,5.0,86.0,34.0,39.5,16.0,44.0,26.0,5.0,1.0,0.0,4.0,11,NaN,7.0,1.0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,38.0,30.0,2.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,7,6.0,4.0,3.0,2.0,7.0,28.6,5.0,114.0,27.0,23.7,27.0,50.0,37.0,9.0,2.0,0.0,7.0,12,NaN,9.0,0.0
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,25.0,18.0,1.0,1.0,3.0,0.0,2.0,7.0,3.0,1.0,0.0,2.0,0.0,1.0,17.0,8,5.0,10.0,2.0,10.0,14.0,71.4,4.0,134.0,38.0,28.4,32.0,70.0,32.0,15.0,4.0,0.0,11.0,15,NaN,12.0,0.0
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.1,0.4,63,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,26.0,21.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,10,11.0,6.0,0.0,4.0,13.0,30.8,9.0,121.0,31.0,25.6,33.0,62.0,26.0,14.0,6.0,0.0,8.0,14,NaN,22.0,0.0
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,NaN,NaN,79,30959.0,Kevin De Bruyne,4-3-3,Robert Jones,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelse

In [74]:
e

,Date,GK_Perf_SoTA,GK_Perf_GA,GK_Perf_Saves,GK_Perf_Save%,GK_Perf_CS,GK_Perf_PSxG,GK_Perf_PSxG+/-,GK_Penalty_PKatt,GK_Penalty_PKA,GK_Penalty_PKsv,GK_Penalty_PKm,GK_Launch_Cmp,GK_Launch_Att,GK_Launch_Cmp%,GK_Passes_Att,GK_Passes_Thr,GK_Passes_Launch%,GK_Passes_AvgLen,Gk_Goal_Kk_Att,Gk_Goal_Kk_Launch%,Gk_Goal_Kk_AvgLen,GK_Crosses_Opp,GK_Crosses_Stp,GK_Crosses_Stp%,Gk_Sweeper_#OPA,Gk_Sweeper_AvgDist
0,2021-08-07,4,1,4,100.0,0,NaN,NaN,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,2,1,1,50.0,0,0.6,-0.4,0,0,0,0,4.0,6.0,66.7,16.0,6.0,31.3,34.9,6.0,16.7,35.2,3.0,0.0,0.0,2.0,21.4
2,2021-08-21,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,3.0,100.0,10.0,3.0,30.0,27.6,2.0,0.0,29.5,4.0,0.0,0.0,2.0,17.6
3,2021-08-28,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,7.0,42.9,33.0,2.0,18.2,27.2,4.0,25.0,38.3,2.0,0.0,0.0,1.0,16.5
4,2021-09-11,1,0,1,100.0,1,0.3,0.3,0,0,0,0,2.0,4.0,50.0,18.0,7.0,22.2,31.2,6.0,0.0,25.7,6.0,0.0,0.0,1.0,14.0
5,2021-09-15,3,3,0,0.0,0,0.9,-2.1,0,0,0,0,3.0,8.0,37.5,32.0,1.0,18.8,29.2,8.0,25.0,25.6,3.0,0.0,0.0,0.0,11.5
6,2021-09-18,2,0,2,100.0,1,0.1,0.1,0,0,0,0,4.0,4.0,100.0,34.0,8.0,8.8,21.6,5.0,20.0,25.0,5.0,0.0,0.0,1.0,18.2
7,2021-09-21,4,1,3,75.0,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-25,0,0,0,NaN,1,0.0,0.0,0,0,0,0,4.0,10.0,40.0,28.0,4.0,28.6,37.2,3.0,66.7,58.0,9.0,1.0,11.1,1.0,16.4
9,2021-09-28,3,2,1,33.3,0,1.4,-0.6,0,0,0,0,1.0,1.0,100.0,10.0,1.0,10.0,21.4,2.0,0.0,21.0,1.0,0.0,0.0,0.0,NaN


In [77]:
abcde = abcd.merge(e, on="Date")

In [78]:
abcde

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err,GK_Perf_SoTA,GK_Perf_GA,GK_Perf_Saves,GK_Perf_Save%,GK_Perf_CS,GK_Perf_PSxG,GK_Perf_PSxG+/-,GK_Penalty_PKatt,GK_Penalty_PKA,GK_Penalty_PKsv,GK_Penalty_PKm,GK_Launch_Cmp,GK_Launch_Att,GK_Launch_Cmp%,GK_Passes_Att,GK_Passes_Thr,GK_Passes_Launch%,GK_Passes_AvgLen,Gk_Goal_Kk_Att,Gk_Goal_Kk_Launch%,Gk_Goal_Kk_AvgLen,GK_Crosses_Opp,GK_Crosses_Stp,GK_Crosses_Stp%,Gk_Sweeper_#OPA,Gk_Sweeper_AvgDist
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,4,1,4,100.0,0,NaN,NaN,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0,2,1,1,50.0,0,0.6,-0.4,0,0,0,0,4.0,6.0,66.7,16.0,6.0,31.3,34.9,6.0,16.7,35.2,3.0,0.0,0.0,2.0,21.4
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,3.0,100.0,10.0,3.0,30.0,27.6,2.0,0.0,29.5,4.0,0.0,0.0,2.0,17.6
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,41.0,39.0,1.0,1.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,11.0,8,2.0,7.0,2.0,5.0,10.0,50.0,5.0,86.0,34.0,39.5,16.0,44.0,26.0,5.0,1.0,0.0,4.0,11,NaN,7.0,1.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,7.0,42.9,33.0,2.0,18.2,27.2,4.0,25.0,38.3,2.0,0.0,0.0,1.0,16.5
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,38.0,30.0,2.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,13.0,7,6.0,4.0,3.0,2.0,7.0,28.6,5.0,114.0,27.0,23.7,27.0,50.0,37.0,9.0,2.0,0.0,7.0,12,NaN,9.0,0.0,1,0,1,100.0,1,0.3,0.3,0,0,0,0,2.0,4.0,50.0,18.0,7.0,22.2,31.2,6.0,0.0,25.7,6.0,0.0,0.0,1.0,14.0
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,25.0,18.0,1.0,1.0,3.0,0.0,2.0,7.0,3.0,1.0,0.0,2.0,0.0,1.0,17.0,8,5.0,10.0,2.0,10.0,14.0,71.4,4.0,134.0,38.0,28.

In [79]:
f

,Date,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_PrgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing_Ast,Passing_xA,Passing_KP,Passing_1/3,Passing_PPA,Passing_CrsPA,Passing_Prog
0,NaN,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331


In [80]:
f['Team'] = team_name
f.drop(['Date'],axis=1,inplace=True)

In [81]:
f

,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_PrgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing_Ast,Passing_xA,Passing_KP,Passing_1/3,Passing_PPA,Passing_CrsPA,Passing_Prog,Team
0,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,Manchester City


In [82]:
abcdef = abcde.merge(f, on="Team")

In [83]:
abcdef

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err,GK_Perf_SoTA,GK_Perf_GA,GK_Perf_Saves,GK_Perf_Save%,GK_Perf_CS,GK_Perf_PSxG,GK_Perf_PSxG+/-,GK_Penalty_PKatt,GK_Penalty_PKA,GK_Penalty_PKsv,GK_Penalty_PKm,GK_Launch_Cmp,GK_Launch_Att,GK_Launch_Cmp%,GK_Passes_Att,GK_Passes_Thr,GK_Passes_Launch%,GK_Passes_AvgLen,Gk_Goal_Kk_Att,Gk_Goal_Kk_Launch%,Gk_Goal_Kk_AvgLen,GK_Crosses_Opp,GK_Crosses_Stp,GK_Crosses_Stp%,Gk_Sweeper_#OPA,Gk_Sweeper_AvgDist,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_PrgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing_Ast,Passing_xA,Passing_KP,Passing_1/3,Passing_PPA,Passing_CrsPA,Passing_Prog
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,4,1,4,100.0,0,NaN,NaN,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0,2,1,1,50.0,0,0.6,-0.4,0,0,0,0,4.0,6.0,66.7,16.0,6.0,31.3,34.9,6.0,16.7,35.2,3.0,0.0,0.0,2.0,21.4,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,3.0,100.0,10.0,3.0,30.0,27.6,2.0,0.0,29.5,4.0,0.0,0.0,2.0,17.6,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,41.0,39.0,1.0,1.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,11.0,8,2.0,7.0,2.0,5.0,10.0,50.0,5.0,86.0,34.0,39.5,16.0,44.0,26.0,5.0,1.0,0.0,4.0,11,NaN,7.0,1.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,7.0,42.9,33.0,2.0,18.2,27.2,4.0,25.0,38.3,2.0,0.0,0.0,1.0,16.5,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331
4,2021-09-11,15:00,Premier Le

In [87]:
g

,Date,Pass_Att,PassType_Live,PassType_Dead,PassType_FK,PassType_TB,PassType_Press,PassType_Sw,PassType_Crs,PassType_CK,Corner_In,Corner_Out,Corner_Str,Pass_Height_Ground,Pass_Height_Low,Pass_Height_High,PassWith_Left,PassWith_Right,PassWith_Head,PassWith_TI,PassWith_Other,Pass_Outome_Cmp,Pass_Outome_Off,Pass_Outome_Out,Pass_Outome_Int,Pass_Outome_Blocks
0,2021-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,569.0,520.0,49.0,13.0,0.0,48.0,18.0,19,11.0,2.0,3.0,2.0,415.0,72.0,82.0,226.0,286.0,22.0,17.0,6.0,476.0,0.0,3.0,22.0,14.0
2,2021-08-21,762.0,724.0,38.0,9.0,1.0,54.0,20.0,19,6.0,4.0,0.0,1.0,610.0,65.0,87.0,221.0,491.0,16.0,20.0,6.0,695.0,1.0,7.0,12.0,13.0
3,2021-08-28,808.0,756.0,52.0,10.0,3.0,65.0,10.0,20,14.0,4.0,1.0,1.0,672.0,52.0,84.0,251.0,507.0,20.0,23.0,2.0,717.0,2.0,11.0,15.0,12.0
4,2021-09-11,603.0,562.0,41.0,6.0,3.0,108.0,16.0,19,8.0,5.0,2.0,0.0,461.0,63.0,79.0,171.0,380.0,20.0,20.0,7.0,525.0,4.0,6.0,10.0,15.0
5,2021-09-15,493.0,442.0,51.0,19.0,2.0,121.0,17.0,7,6.0,2.0,2.0,0.0,367.0,50.0,76.0,194.0,263.0,12.0,14.0,5.0,413.0,5.0,6.0,14.0,11.0
6,2021-09-18,600.0,554.0,46.0,11.0,0.0,56.0,13.0,19,8.0,3.0,3.0,0.0,441.0,89.0,70.0,178.0,357.0,14.0,21.0,15.0,512.0,3.0,5.0,19.0,12.0
7,2021-09-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-25,635.0,591.0,44.0,15.0,1.0,101.0,18.0,15,13.0,8.0,1.0,1.0,505.0,48.0,82.0,209.0,386.0,12.0,12.0,6.0,544.0,2.0,11.0,11.0,15.0
9,2021-09-28,589.0,540.0,49.0,9.0,0.0,73.0,29.0,24,10.0,7.0,0.0,0.0,444.0,75.0,70.0,203.0,348.0,9.0,25.0,1.0,507.0,3.0,9.0,18.0,10.0


In [88]:
abcdefg = abcdef.merge(g, on="Date")

In [89]:
abcdefg

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err,GK_Perf_SoTA,GK_Perf_GA,GK_Perf_Saves,GK_Perf_Save%,GK_Perf_CS,GK_Perf_PSxG,GK_Perf_PSxG+/-,GK_Penalty_PKatt,GK_Penalty_PKA,GK_Penalty_PKsv,GK_Penalty_PKm,GK_Launch_Cmp,GK_Launch_Att,GK_Launch_Cmp%,GK_Passes_Att,GK_Passes_Thr,GK_Passes_Launch%,GK_Passes_AvgLen,Gk_Goal_Kk_Att,Gk_Goal_Kk_Launch%,Gk_Goal_Kk_AvgLen,GK_Crosses_Opp,GK_Crosses_Stp,GK_Crosses_Stp%,Gk_Sweeper_#OPA,Gk_Sweeper_AvgDist,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_PrgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing_Ast,Passing_xA,Passing_KP,Passing_1/3,Passing_PPA,Passing_CrsPA,Passing_Prog,Pass_Att,PassType_Live,PassType_Dead,PassType_FK,PassType_TB,PassType_Press,PassType_Sw,PassType_Crs,PassType_CK,Corner_In,Corner_Out,Corner_Str,Pass_Height_Ground,Pass_Height_Low,Pass_Height_High,PassWith_Left,PassWith_Right,PassWith_Head,PassWith_TI,PassWith_Other,Pass_Outome_Cmp,Pass_Outome_Off,Pass_Outome_Out,Pass_Outome_Int,Pass_Outome_Blocks
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,4,1,4,100.0,0,NaN,NaN,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0,2,1,1,50.0,0,0.6,-0.4,0,0,0,0,4.0,6.0,66.7,16.0,6.0,31.3,34.9,6.0,16.7,35.2,3.0,0.0,0.0,2.0,21.4,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,569.0,520.0,49.0,13.0,0.0,48.0,18.0,19,11.0,2.0,3.0,2.0,415.0,72.0,82.0,226.0,286.0,22.0,17.0,6.0,476.0,0.0,3.0,22.0,14.0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,3.0,100.0,10.0,3.0,30.0,27.6,2.0,0.0,29.5,4.0,0.0,0.0,2.0,17.6,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,762.0,724.0,38.0,9.0,1.0,54.0,20.0,19,6.0,4.0,0

In [90]:
h

,Date,Poss,Touches_Touches,Touches_Def_Pen,Touches_Def_3rd,Touches_Mid_3rd,Touches_Att_3rd,Touches_Att_Pen,Touches_Live,Dribbles_Succ,Dribbles_Att,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_Prog,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2021-08-07,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-15,64.0,676.0,38.0,148.0,360.0,227.0,43.0,629.0,12.0,19.0,63.2,12.0,2.0,491.0,2689.0,1702.0,79.0,23.0,12.0,10.0,15.0,542.0,476.0,87.8,45.0
2,2021-08-21,67.0,850.0,26.0,80.0,485.0,328.0,61.0,813.0,9.0,15.0,60.0,11.0,0.0,674.0,3425.0,1872.0,102.0,14.0,19.0,9.0,12.0,737.0,695.0,94.3,57.0
3,2021-08-28,80.0,879.0,43.0,139.0,453.0,320.0,52.0,829.0,8.0,14.0,57.1,8.0,1.0,677.0,2846.0,1795.0,98.0,18.0,13.0,7.0,3.0,792.0,717.0,90.5,47.0
4,2021-09-11,61.0,707.0,38.0,157.0,371.0,241.0,51.0,666.0,7.0,16.0,43.8,9.0,1.0,480.0,2887.0,1712.0,77.0,18.0,11.0,9.0,11.0,572.0,525.0,91.8,58.0
5,2021-09-15,51.0,603.0,76.0,226.0,307.0,118.0,26.0,555.0,7.0,19.0,36.8,7.0,1.0,361.0,2018.0,1224.0,40.0,7.0,10.0,10.0,5.0,471.0,413.0,87.7,32.0
6,2021-09-18,63.0,720.0,74.0,226.0,382.0,163.0,31.0,673.0,12.0,20.0,60.0,12.0,1.0,530.0,2390.0,1378.0,49.0,12.0,8.0,18.0,10.0,575.0,512.0,89.0,39.0
7,2021-09-21,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-25,60.0,751.0,69.0,181.0,397.0,221.0,37.0,708.0,11.0,18.0,61.1,12.0,2.0,573.0,2819.0,1620.0,84.0,20.0,12.0,10.0,9.0,621.0,544.0,87.6,34.0
9,2021-09-28,54.0,684.0,33.0,137.0,275.0,317.0,47.0,637.0,11.0,16.0,68.8,11.0,0.0,503.0,2930.0,1865.0,87.0,20.0,15.0,5.0,16.0,562.0,507.0,90.2,58.0


In [93]:
abcdefgh = abcdef.merge(h, on="Date")

In [96]:
abcdefghi = abcdef.merge(i, on="Date")

In [97]:
abcdefghi

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,SCA_Types_SCA,SCA_Types_PassLive,SCA_Types_PassDead,SCA_Types_Drib,SCA_Types_Sh,SCA_Types_Fld,SCA_Types_Def,GCA_Types_GCA,GCA_Types_PassLive,GCA_Types_PassDead,GCA_Types_Drib,GCA_Types_Sh,GCA_Types_Fld,GCA_Types_Def,Tackles_Tkl,Tackles_TklW,Tackles_Def_3rd,Tackles_Mid_3rd,Tackles_Att_3rd,Vs_Dribbles_Tkl,Vs_Dribbles_Att,Vs_Dribbles_Tkl%,Vs_Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def_3rd,Pressures_Mid_3rd,Pressures_Att_3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,Def_Int,Def_Tkl+Int,Def_Clr,Def_Err,GK_Perf_SoTA,GK_Perf_GA,GK_Perf_Saves,GK_Perf_Save%,GK_Perf_CS,GK_Perf_PSxG,GK_Perf_PSxG+/-,GK_Penalty_PKatt,GK_Penalty_PKA,GK_Penalty_PKsv,GK_Penalty_PKm,GK_Launch_Cmp,GK_Launch_Att,GK_Launch_Cmp%,GK_Passes_Att,GK_Passes_Thr,GK_Passes_Launch%,GK_Passes_AvgLen,Gk_Goal_Kk_Att,Gk_Goal_Kk_Launch%,Gk_Goal_Kk_AvgLen,GK_Crosses_Opp,GK_Crosses_Stp,GK_Crosses_Stp%,Gk_Sweeper_#OPA,Gk_Sweeper_AvgDist,Passing_Total_Cmp,Passing_Total_Att,Passing_Total_Cmp%,Passing_Total_TotDist,Passing_Total_PrgDist,Passing_Short_Cmp,Passing_Short_Att,Passing_Short_Cmp%,Passing_Medium_Cmp,Passing_Medium_Att,Passing_Medium_Cmp%,Passing_Long_Cmp,Passing_Long_Att,Passing_Long_Cmp%,Passing_Ast,Passing_xA,Passing_KP,Passing_1/3,Passing_PPA,Passing_CrsPA,Passing_Prog,Misc_Stats_CrdY,Misc_Stats_CrdR,Misc_Stats_2CrdY,Misc_Stats_Fls,Misc_Stats_Fld,Misc_Stats_Off,Misc_Stats_Crs,Misc_Stats_Int,Misc_Stats_TklW,Misc_Stats_PKwon,Misc_Stats_PKcon,Misc_Stats_OG,Misc_Stats_Recov,Misc_Stats_Aerial_DuelsWon,Misc_Stats_Aerial_DuelsLost,Misc_Stats_Aerial_DuelsWon%
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,4,1,4,100.0,0,NaN,NaN,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,2,0,0,8,6,3,12,3,7,NaN,NaN,0,NaN,NaN,NaN,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,31.0,24.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,10,6.0,10.0,2.0,4.0,15.0,26.7,11.0,58.0,22.0,37.9,16.0,29.0,13.0,12.0,6.0,1.0,6.0,6,NaN,17.0,1.0,2,1,1,50.0,0,0.6,-0.4,0,0,0,0,4.0,6.0,66.7,16.0,6.0,31.3,34.9,6.0,16.7,35.2,3.0,0.0,0.0,2.0,21.4,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,1,0,0,12,12,1,19,6,10,0.0,0.0,0,90.0,19.0,13.0,59.4
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.37,15.7,27,11,14,113.1,104.6,0.12,32.9,30.4,21.0,15.0,2.0,0.0,1.0,1.0,2.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,15.0,10,2.0,8.0,5.0,10.0,19.0,52.6,9.0,115.0,32.0,27.8,4.0,52.0,59.0,4.0,0.0,0.0,4.0,10,NaN,4.0,0.0,0,0,0,NaN,1,0.0,0.0,0,0,0,0,3.0,3.0,100.0,10.0,3.0,30.0,27.6,2.0,0.0,29.5,4.0,0.0,0.0,2.0,17.6,31037,35247,88.1,590100,156784,13000,14018,92.7,13648,14862,91.8,4036,5458,73.9,102,77.3,676,2552,726,100,2331,1,0,0,14,6,2,19,10,10,0.0,0.0,0,71.0,9.0,1.0,90.0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,Manchester City,146,1052,361,34.3,0.13,0.